In [75]:
import pandas as pd
import numpy as np
import requests
import regex as re
from bs4 import BeautifulSoup
from tweetscrape.search_tweets import TweetScrapperSearch
from tweetscrape.profile_tweets import TweetScrapperProfile

setting topic and number of tweets to scrape

In [8]:
topic = 'coronavirus'
number = 40

Scraping the tweets and saving them as a csv to twitter.csv in Data folder

In [13]:
tweet_scrapper = TweetScrapperSearch(topic, num_tweets = number, tweet_dump_path='../Data/twitter.csv', tweet_dump_format = 'csv')
tweet_count, tweet_id, tweet_time, dump_path = tweet_scrapper.get_search_tweets()
print("Extracted {0} tweets till {1} at {2}".format(tweet_count, tweet_time, dump_path))

Extracted 40 tweets till 2019-10-23 at twitter2.csv


# EXTRACTING DATA FROM 4 BIG INTERNATIONALISTS

# 1. BBC 

### BBC. Getting all the data from the BBC search

In [34]:
bbc_url = 'https://www.bbc.co.uk/search?q=coronavirus'
bbc = []
res = requests.get(bbc_url)
soup = BeautifulSoup(res.content, 'html.parser')
bbc_headlines = []

for h in soup.find_all('h1'):
    a = h.find('a')
    bbc_dict = {'headline' : h.text,
                'url' : a.attrs['href']}
    bbc_headlines.append(bbc_dict)

bbc = pd.DataFrame(bbc_headlines)

### BBC. Creating a DataFrame from all the text in each story

In [35]:
stories = []
for url in bbc['url'][1:]:
    res = requests.get(url)
    print(res.status_code)
    soup = BeautifulSoup(res.content, 'lxml')
    content = soup.find('div', {'class' : 'story-body__inner'})
    df = pd.DataFrame(content, columns = ['text'])
    df['source'] = url
    stories.append(pd.DataFrame(df))

200
200
200
200
200
200
200
200
200
200


### BBC. Merging the dataframe

In [36]:
df = pd.concat(stories)

### BBC. Stripping the html, retrieving text

In [37]:
def html_stripper(x):
    try:
        return x.text
    except:
        pass

df['dum'] = df['text'].apply(lambda x : html_stripper(x))

df = df[~df['dum'].isna()]

df = df[~df['dum'].astype(str).str.startswith('\n')]

In [38]:
df = df[['dum', 'source']]

In [39]:
df

,dum,source
2,A religious group in South Korea has been iden...,https://www.bbc.co.uk/news/world-asia-51572137
3,The infections among members of the Shincheonj...,https://www.bbc.co.uk/news/world-asia-51572137
4,The sect in the city of Daegu accounts for 30 ...,https://www.bbc.co.uk/news/world-asia-51572137
5,South Korea also confirmed its first death fro...,https://www.bbc.co.uk/news/world-asia-51572137
7,"The coronavirus, which originated in China's H...",https://www.bbc.co.uk/news/world-asia-51572137
...,...,...
50,,https://www.bbc.co.uk/news/health-51048366
52,What questions do you have about the coronavirus?,https://www.bbc.co.uk/news/health-51048366
53,"In some cases your question will be published,...",https://www.bbc.co.uk/news/health-51048366
54,Use this form to ask your question:,https://www.bbc.co.uk/news/health-51048366


### BBC. Merging the text into one block for each story

In [40]:
source = df.groupby('source')

In [41]:
df['source'].unique()

array(['https://www.bbc.co.uk/news/world-asia-51572137',
       'https://www.bbc.co.uk/news/uk-england-oxfordshire-51573339',
       'https://www.bbc.co.uk/news/world-51235105',
       'https://www.bbc.co.uk/news/world-asia-51568496',
       'https://www.bbc.co.uk/news/health-51048366'], dtype=object)

In [42]:
texts = []
for i in df['source'].unique():
    group = source.get_group(i)
    texts.append(pd.DataFrame({'text' : [group['dum'].str.cat(sep = ' ')], 'url':[i]}))
df_text = pd.concat(texts)

In [49]:
df_text.index = [0,1,2,3,4]

In [50]:
df_text

,text,url
0,A religious group in South Korea has been iden...,https://www.bbc.co.uk/news/world-asia-51572137
1,A prisoner at the centre of a coronavirus scar...,https://www.bbc.co.uk/news/uk-england-oxfordsh...
2,"A fast-moving virus known as the ""new coronavi...",https://www.bbc.co.uk/news/world-51235105
3,Two Japanese passengers who contracted the new...,https://www.bbc.co.uk/news/world-asia-51568496
4,A virus causing severe lung disease that start...,https://www.bbc.co.uk/news/health-51048366


## 2. AP

### AP. Getting all the data from the AP search

In [51]:
apnews_url = 'https://apnews.com/Coronavirus'
ap =[]
res1 = requests.get(apnews_url)
soup = BeautifulSoup(res1.content, 'html.parser')
ap_headlines = []

for a in soup.find_all('a'):
    for h in a.find_all('h1'):
        ap_dict = {'headline' : h.text,
                   'url' : 'https://apnews.com/Coronavirus' + a.attrs['href']}
        ap_headlines.append(ap_dict)
        
ap = pd.DataFrame(ap_headlines)

In [52]:
ap.head()

,headline,url
0,New Hampshire urges recent arrivals from China...,https://apnews.com/Coronavirus/b118518ec4d1e43...
1,"New virus has infected more than 14,550 globally",https://apnews.com/Coronavirus/a2ce1ce323858e7...
2,Apple temporarily closes stores in China amid ...,https://apnews.com/Coronavirus/bac0f4b76cf8bfe...
3,"Chinese communities rattled by virus, cancel n...",https://apnews.com/Coronavirus/c14aa16d2351f6e...
4,Man United fills striker void by signing Ighal...,https://apnews.com/Coronavirus/1d6e3def5b52806...


### AP. Creating a DataFrame from all the text in each story

In [53]:
stories = []
urls = []
for i in ap['url']:
    print(i)
    res = requests.get(i)
    print(res.status_code)
    soup = BeautifulSoup(res.content, 'lxml')
    content = soup.find('div', {'class': 'Article'})
    stories.append(content)
    urls.append(i)

https://apnews.com/Coronavirus/b118518ec4d1e434b17135c04e195bd7
200
https://apnews.com/Coronavirus/a2ce1ce323858e798bb985b9ef242f32
200
https://apnews.com/Coronavirus/bac0f4b76cf8bfe10204d132c6ddff81
200
https://apnews.com/Coronavirus/c14aa16d2351f6e7a54c267f5772669b
200
https://apnews.com/Coronavirus/1d6e3def5b5280684448f5110ef84b03
200
https://apnews.com/Coronavirus/7f7336d2ed099936bd59bf8cb7f43756
200
https://apnews.com/Coronavirus/4e654fb27cb4bbdc9618921eaeeabe40
200
https://apnews.com/Coronavirus/6d1045e3bb3ce8959fe7b22a97904a13
200


### AP. Stripping the html, retrieving text

In [54]:
ap_scrapes = pd.DataFrame()
ap_scrapes['text'] = stories
ap_scrapes['source'] = urls
ap_scrapes

,text,source
0,"[[CONCORD, N.H. (AP) — New Hampshire public he...",https://apnews.com/Coronavirus/b118518ec4d1e43...
1,[[A viral outbreak that began in China has inf...,https://apnews.com/Coronavirus/a2ce1ce323858e7...
2,[[SAN FRANCISCO (AP) — Apple is temporarily cl...,https://apnews.com/Coronavirus/bac0f4b76cf8bfe...
3,"[[ALHAMBRA, Calif. (AP) — The rapid spread of ...",https://apnews.com/Coronavirus/c14aa16d2351f6e...
4,[[LONDON (AP) — The coronavirus-enforced suspe...,https://apnews.com/Coronavirus/1d6e3def5b52806...
5,"[[BEIJING (AP) — Meron Mei, a sophomore at Wuh...",https://apnews.com/Coronavirus/7f7336d2ed09993...
6,[[A roundup of some of the most popular but co...,https://apnews.com/Coronavirus/4e654fb27cb4bbd...
7,"[[NEW HAVEN, Conn. (AP) — A high school studen...",https://apnews.com/Coronavirus/6d1045e3bb3ce89...


In [55]:
ap_scrapes['dum'] = ap_scrapes['text'].apply(lambda x : html_stripper(x))

ap_scrapes = ap_scrapes[~ap_scrapes['dum'].isna()]

ap_scrapes = ap_scrapes[~ap_scrapes['dum'].astype(str).str.startswith('\n')]

ap_scrapes = ap_scrapes[['dum', 'source']]
ap_scrapes

,dum,source
0,"CONCORD, N.H. (AP) — New Hampshire public heal...",https://apnews.com/Coronavirus/b118518ec4d1e43...
1,A viral outbreak that began in China has infec...,https://apnews.com/Coronavirus/a2ce1ce323858e7...
2,SAN FRANCISCO (AP) — Apple is temporarily clos...,https://apnews.com/Coronavirus/bac0f4b76cf8bfe...
3,"ALHAMBRA, Calif. (AP) — The rapid spread of a ...",https://apnews.com/Coronavirus/c14aa16d2351f6e...
4,LONDON (AP) — The coronavirus-enforced suspens...,https://apnews.com/Coronavirus/1d6e3def5b52806...
5,"BEIJING (AP) — Meron Mei, a sophomore at Wuhan...",https://apnews.com/Coronavirus/7f7336d2ed09993...
6,A roundup of some of the most popular but comp...,https://apnews.com/Coronavirus/4e654fb27cb4bbd...
7,"NEW HAVEN, Conn. (AP) — A high school student ...",https://apnews.com/Coronavirus/6d1045e3bb3ce89...


### AP. Merging the text into one block for each story

In [56]:
source = ap_scrapes.groupby('source')

In [57]:
ap_scrapes['source'].unique()

array(['https://apnews.com/Coronavirus/b118518ec4d1e434b17135c04e195bd7',
       'https://apnews.com/Coronavirus/a2ce1ce323858e798bb985b9ef242f32',
       'https://apnews.com/Coronavirus/bac0f4b76cf8bfe10204d132c6ddff81',
       'https://apnews.com/Coronavirus/c14aa16d2351f6e7a54c267f5772669b',
       'https://apnews.com/Coronavirus/1d6e3def5b5280684448f5110ef84b03',
       'https://apnews.com/Coronavirus/7f7336d2ed099936bd59bf8cb7f43756',
       'https://apnews.com/Coronavirus/4e654fb27cb4bbdc9618921eaeeabe40',
       'https://apnews.com/Coronavirus/6d1045e3bb3ce8959fe7b22a97904a13'],
      dtype=object)

In [58]:
texts = []
for i in ap_scrapes['source'].unique():
    group = source.get_group(i)
    texts.append(pd.DataFrame({'text' : [group['dum'].str.cat(sep = ' ')], 'url':[i]}))
ap_text = pd.concat(texts)

In [59]:
ap_text

,text,url
0,"CONCORD, N.H. (AP) — New Hampshire public heal...",https://apnews.com/Coronavirus/b118518ec4d1e43...
0,A viral outbreak that began in China has infec...,https://apnews.com/Coronavirus/a2ce1ce323858e7...
0,SAN FRANCISCO (AP) — Apple is temporarily clos...,https://apnews.com/Coronavirus/bac0f4b76cf8bfe...
0,"ALHAMBRA, Calif. (AP) — The rapid spread of a ...",https://apnews.com/Coronavirus/c14aa16d2351f6e...
0,LONDON (AP) — The coronavirus-enforced suspens...,https://apnews.com/Coronavirus/1d6e3def5b52806...
0,"BEIJING (AP) — Meron Mei, a sophomore at Wuhan...",https://apnews.com/Coronavirus/7f7336d2ed09993...
0,A roundup of some of the most popular but comp...,https://apnews.com/Coronavirus/4e654fb27cb4bbd...
0,"NEW HAVEN, Conn. (AP) — A high school student ...",https://apnews.com/Coronavirus/6d1045e3bb3ce89...


## 3. Reuters

### Reuters. Getting all the data from the Reuters search

In [60]:
reunews_url = 'https://www.reuters.com/search/news?blob=coronavirus'

rue =[]

res2 = requests.get(reunews_url)

soup = BeautifulSoup(res2.content, 'html.parser')

rue_headlines = []

for h in soup.find_all('h3'):
    try:
        a = h.find('a')
        rue_dict = {'headline' : h.text,
                    'url' : 'https://www.reuters.com' + a.attrs['href']}
        rue_headlines.append(rue_dict)
    except:
        pass
        
rue = pd.DataFrame(rue_headlines)

In [61]:
rue.head()

,headline,url
0,Start of PCS postponed due to coronavirus,https://www.reuters.com/article/idUSFLM77stW1
1,"Dynasty, Gen.G to donate to coronavirus charities",https://www.reuters.com/article/idUSFLM1lGRY3
2,Reuters coronavirus coverage leads the way,https://www.reuters.com/article/idUSKBN20B1J0
3,France confirms sixth case of infection with c...,https://www.reuters.com/article/idUSKBN1ZT2OE
4,Gilead assessing Ebola drug as possible corona...,https://www.reuters.com/article/idUSL4N29S45D


### Reuters. Creating a DataFrame from all the text in each story

In [62]:
stories = []
urls = []
for i in rue['url']:
    print(i)
    res2 = requests.get(i)
    print(res2.status_code)
    soup = BeautifulSoup(res2.content, 'lxml')
    content = soup.find('div', {'class': 'StandardArticleBody_body'})
    stories.append(content)
    urls.append(i)

https://www.reuters.com/article/idUSFLM77stW1
200
https://www.reuters.com/article/idUSFLM1lGRY3
200
https://www.reuters.com/article/idUSKBN20B1J0
200
https://www.reuters.com/article/idUSKBN1ZT2OE
200
https://www.reuters.com/article/idUSL4N29S45D
200
https://www.reuters.com/article/idUSFWN2AC19O
200
https://www.reuters.com/article/idUSP6N28L01C
200
https://www.reuters.com/article/idUSP6N28L01K
200
https://www.reuters.com/article/idUSL4N29Y2NM
200
https://www.reuters.com/article/idUSKBN1ZU2H0
200


In [63]:
rue_scrapes = pd.DataFrame()
rue_scrapes['text'] = stories
rue_scrapes['source'] = urls
rue_scrapes

,text,source
0,[[The start of the League of Legends Pacific C...,https://www.reuters.com/article/idUSFLM77stW1
1,[[The Seoul Dynasty and Gen.G Esports will don...,https://www.reuters.com/article/idUSFLM1lGRY3
2,[[Reuters coverage of the coronavirus continue...,https://www.reuters.com/article/idUSKBN20B1J0
3,"[[[<figure class=""Image_zoom"" style=""padding-b...",https://www.reuters.com/article/idUSKBN1ZT2OE
4,[[Jan 23 (Reuters) - Gilead Sciences Inc said ...,https://www.reuters.com/article/idUSL4N29S45D
5,"[[Feb 13 (Reuters) - Vmoto Ltd: ], [* VMOTO LT...",https://www.reuters.com/article/idUSFWN2AC19O
6,"[[PARIS, Jan 29 (Reuters) - France has confirm...",https://www.reuters.com/article/idUSP6N28L01C
7,"[[PARIS, Jan 30 (Reuters) - France has confirm...",https://www.reuters.com/article/idUSP6N28L01K
8,"[[LONDON, Jan 29 (Reuters) - Here are some key...",https://www.reuters.com/article/idUSL4N29Y2NM
9,[[MOSCOW (Reuters) - Russia is ready to bring ...,https://www.reuters.com/article/idUSKBN1ZU2H0


### Reuters. Stripping the html, retrieving text

In [64]:
rue_scrapes['dum'] = rue_scrapes['text'].apply(lambda x : html_stripper(x))

rue_scrapes = rue_scrapes[~rue_scrapes['dum'].isna()]

rue_scrapes = rue_scrapes[~rue_scrapes['dum'].astype(str).str.startswith('\n')]

rue_scrapes = rue_scrapes[['dum', 'source']]
rue_scrapes

,dum,source
0,The start of the League of Legends Pacific Cha...,https://www.reuters.com/article/idUSFLM77stW1
1,The Seoul Dynasty and Gen.G Esports will donat...,https://www.reuters.com/article/idUSFLM1lGRY3
2,Reuters coverage of the coronavirus continues ...,https://www.reuters.com/article/idUSKBN20B1J0
3,A tourist wears a protective mask as she leave...,https://www.reuters.com/article/idUSKBN1ZT2OE
4,Jan 23 (Reuters) - Gilead Sciences Inc said on...,https://www.reuters.com/article/idUSL4N29S45D
5,Feb 13 (Reuters) - Vmoto Ltd: * VMOTO LTD- UP...,https://www.reuters.com/article/idUSFWN2AC19O
6,"PARIS, Jan 29 (Reuters) - France has confirmed...",https://www.reuters.com/article/idUSP6N28L01C
7,"PARIS, Jan 30 (Reuters) - France has confirmed...",https://www.reuters.com/article/idUSP6N28L01K
8,"LONDON, Jan 29 (Reuters) - Here are some key f...",https://www.reuters.com/article/idUSL4N29Y2NM
9,MOSCOW (Reuters) - Russia is ready to bring fo...,https://www.reuters.com/article/idUSKBN1ZU2H0


### Rueters. Merging the text into one block for each story¶

In [65]:
source = rue_scrapes.groupby('source')

In [66]:
rue_scrapes['source'].unique()

array(['https://www.reuters.com/article/idUSFLM77stW1',
       'https://www.reuters.com/article/idUSFLM1lGRY3',
       'https://www.reuters.com/article/idUSKBN20B1J0',
       'https://www.reuters.com/article/idUSKBN1ZT2OE',
       'https://www.reuters.com/article/idUSL4N29S45D',
       'https://www.reuters.com/article/idUSFWN2AC19O',
       'https://www.reuters.com/article/idUSP6N28L01C',
       'https://www.reuters.com/article/idUSP6N28L01K',
       'https://www.reuters.com/article/idUSL4N29Y2NM',
       'https://www.reuters.com/article/idUSKBN1ZU2H0'], dtype=object)

In [67]:
texts = []
for i in rue_scrapes['source'].unique():
    group = source.get_group(i)
    texts.append(pd.DataFrame({'text' : [group['dum'].str.cat(sep = ' ')], 'url':[i]}))
rue_text = pd.concat(texts)

In [68]:
rue_text

,text,url
0,The start of the League of Legends Pacific Cha...,https://www.reuters.com/article/idUSFLM77stW1
0,The Seoul Dynasty and Gen.G Esports will donat...,https://www.reuters.com/article/idUSFLM1lGRY3
0,Reuters coverage of the coronavirus continues ...,https://www.reuters.com/article/idUSKBN20B1J0
0,A tourist wears a protective mask as she leave...,https://www.reuters.com/article/idUSKBN1ZT2OE
0,Jan 23 (Reuters) - Gilead Sciences Inc said on...,https://www.reuters.com/article/idUSL4N29S45D
0,Feb 13 (Reuters) - Vmoto Ltd: * VMOTO LTD- UP...,https://www.reuters.com/article/idUSFWN2AC19O
0,"PARIS, Jan 29 (Reuters) - France has confirmed...",https://www.reuters.com/article/idUSP6N28L01C
0,"PARIS, Jan 30 (Reuters) - France has confirmed...",https://www.reuters.com/article/idUSP6N28L01K
0,"LONDON, Jan 29 (Reuters) - Here are some key f...",https://www.reuters.com/article/idUSL4N29Y2NM
0,MOSCOW (Reuters) - Russia is ready to bring fo...,https://www.reuters.com/article/idUSKBN1ZU2H0


## 4. UPI 

### UPI. Getting all the data from the UPI search

In [69]:
upinews_url = 'https://www.upi.com/search?ss=coronavirus'
upi =[]
res3 = requests.get(upinews_url)
soup = BeautifulSoup(res3.content, 'html.parser')
upi_headlines = []
row_story = soup.find('div', {"class": "row story list"})
for x in row_story.find_all('a'):
    upi_dict ={'headline' : x.attrs['title'],
            'url' : x.attrs['href']}
    upi_headlines.append(upi_dict)
upi = pd.DataFrame(upi_headlines)

In [70]:
upi.head()

,headline,url
0,"Coronavirus outbreak slows, research into drug...",https://www.upi.com/Health_News/2020/02/20/Cor...
1,Prices rising in North Korea following COVID-1...,https://www.upi.com/Top_News/World-News/2020/0...
2,Forum explores challenges in travel to North K...,https://www.upi.com/Top_News/World-News/2020/0...
3,South Korea reports first COVID-19 death; case...,https://www.upi.com/Top_News/World-News/2020/0...
4,2 passengers of coronavirus-infected ship die ...,https://www.upi.com/Top_News/World-News/2020/0...


### UPI. Creating a DataFrame from all the text in each story

In [71]:
stories = []
urls = []
for i in upi['url']:
    print(i)
    res = requests.get(i)
    print(res.status_code)
    soup = BeautifulSoup(res.content, 'lxml')
    content = soup.find('article', {'itemprop': 'articleBody'})
    stories.append(content)
    urls.append(i)

https://www.upi.com/Health_News/2020/02/20/Coronavirus-outbreak-slows-research-into-drug-treatments-speeds-up/9091582211534/?upi_ss=coronavirus
200
https://www.upi.com/Top_News/World-News/2020/02/20/Prices-rising-in-North-Korea-following-COVID-19-shutdown/9141582218258/?upi_ss=coronavirus
200
https://www.upi.com/Top_News/World-News/2020/02/20/Forum-explores-challenges-in-travel-to-North-Korea/4301582205079/?upi_ss=coronavirus
200
https://www.upi.com/Top_News/World-News/2020/02/20/South-Korea-reports-first-COVID-19-death-cases-rise-to-104/6161582193510/?upi_ss=coronavirus
200
https://www.upi.com/Top_News/World-News/2020/02/20/2-passengers-of-coronavirus-infected-ship-die-in-Japan/8981582185585/?upi_ss=coronavirus
200
https://www.upi.com/Top_News/World-News/2020/02/19/North-Korea-charges-Souths-entertainment-with-destroying-language/8811582135340/?upi_ss=coronavirus
200
https://www.upi.com/Top_News/World-News/2020/02/19/Mass-animal-die-off-in-China-due-to-disinfectants-authorities-say/35

In [72]:
upi_scrapes = pd.DataFrame()
upi_scrapes['text'] = stories
upi_scrapes['source'] = urls
upi_scrapes

,text,source
0,"[\n, [[Feb. 20 (UPI) --], As fast as the nove...",https://www.upi.com/Health_News/2020/02/20/Cor...
1,"[\n, [[Feb. 20 (UPI) --], Prices may be risin...",https://www.upi.com/Top_News/World-News/2020/0...
2,"[\n, [[SEOUL, Feb. 20 (UPI) --], Experts, civ...",https://www.upi.com/Top_News/World-News/2020/0...
3,"[\n, [[SEOUL, Feb. 20 (UPI) --], South Korean...",https://www.upi.com/Top_News/World-News/2020/0...
4,"[\n, [[Feb. 20 (UPI) --], Two passengers of a...",https://www.upi.com/Top_News/World-News/2020/0...
5,"[\n, [[Feb. 19 (UPI) --], , [North Korea], s...",https://www.upi.com/Top_News/World-News/2020/0...
6,"[\n, [[Feb. 19 (UPI) --], China is using all ...",https://www.upi.com/Top_News/World-News/2020/0...
7,"[\n, [[Feb. 19 (UPI) --], Egypt, Algeria and ...",https://www.upi.com/Health_News/2020/02/19/Egy...
8,"[\n, [[Feb. 19 (UPI) --], South Korea reporte...",https://www.upi.com/Top_News/World-News/2020/0...
9,"[\n, [In my book, , [Anatomy of Failure: Why A...",https://www.upi.com/Top_News/Voices/2020/02/19...


### UPI. Stripping the html, retrieving text

In [78]:
upi_scrapes['dum'] = upi_scrapes['text'].apply(lambda x : html_stripper(x))
upi_scrapes['dum'] = upi_scrapes['dum'].str.replace('\n', '')
upi_scrapes['dum'] = upi_scrapes['dum'].apply(lambda x:re.sub('.*[\(]UPI[\)]\s--\s', '', x))


In [79]:
upi_scrapes

,text,source,dum
0,"[\n, [[Feb. 20 (UPI) --], As fast as the nove...",https://www.upi.com/Health_News/2020/02/20/Cor...,"As fast as the novel coronavirus, or COVID-19,..."
1,"[\n, [[Feb. 20 (UPI) --], Prices may be risin...",https://www.upi.com/Top_News/World-News/2020/0...,Prices may be rising in North Korea as the cou...
2,"[\n, [[SEOUL, Feb. 20 (UPI) --], Experts, civ...",https://www.upi.com/Top_News/World-News/2020/0...,"Experts, civic groups and others gathered in S..."
3,"[\n, [[SEOUL, Feb. 20 (UPI) --], South Korean...",https://www.upi.com/Top_News/World-News/2020/0...,South Korean officials reported the nation's f...
4,"[\n, [[Feb. 20 (UPI) --], Two passengers of a...",https://www.upi.com/Top_News/World-News/2020/0...,Two passengers of a quarantined cruise ship in...
5,"[\n, [[Feb. 19 (UPI) --], , [North Korea], s...",https://www.upi.com/Top_News/World-News/2020/0...,North Korea state media berated South Korean t...
6,"[\n, [[Feb. 19 (UPI) --], China is using all ...",https://www.upi.com/Top_News/World-News/2020/0...,China is using all available measures to stop ...
7,"[\n, [[Feb. 19 (UPI) --], Egypt, Algeria and ...",https://www.upi.com/Health_News/2020/02/19/Egy...,"Egypt, Algeria and South Africa are among the ..."
8,"[\n, [[Feb. 19 (UPI) --], South Korea reporte...",https://www.upi.com/Top_News/World-News/2020/0...,South Korea reported a record number of confir...
9,"[\n, [In my book, , [Anatomy of Failure: Why A...",https://www.upi.com/Top_News/Voices/2020/02/19...,"In my book, Anatomy of Failure: Why America Lo..."


### UPI. Merging the text into one block for each story¶


In [80]:
source = upi_scrapes.groupby('source')

In [81]:
upi_scrapes['source'].unique()

array(['https://www.upi.com/Health_News/2020/02/20/Coronavirus-outbreak-slows-research-into-drug-treatments-speeds-up/9091582211534/?upi_ss=coronavirus',
       'https://www.upi.com/Top_News/World-News/2020/02/20/Prices-rising-in-North-Korea-following-COVID-19-shutdown/9141582218258/?upi_ss=coronavirus',
       'https://www.upi.com/Top_News/World-News/2020/02/20/Forum-explores-challenges-in-travel-to-North-Korea/4301582205079/?upi_ss=coronavirus',
       'https://www.upi.com/Top_News/World-News/2020/02/20/South-Korea-reports-first-COVID-19-death-cases-rise-to-104/6161582193510/?upi_ss=coronavirus',
       'https://www.upi.com/Top_News/World-News/2020/02/20/2-passengers-of-coronavirus-infected-ship-die-in-Japan/8981582185585/?upi_ss=coronavirus',
       'https://www.upi.com/Top_News/World-News/2020/02/19/North-Korea-charges-Souths-entertainment-with-destroying-language/8811582135340/?upi_ss=coronavirus',
       'https://www.upi.com/Top_News/World-News/2020/02/19/Mass-animal-die-off-in-C

In [82]:
texts = []
for i in upi_scrapes['source'].unique():
    group = source.get_group(i)
    texts.append(pd.DataFrame({'text' : [group['dum'].str.cat(sep = ' ')], 'url':[i]}))
upi_text = pd.concat(texts)

In [83]:
upi_text

,text,url
0,"As fast as the novel coronavirus, or COVID-19,...",https://www.upi.com/Health_News/2020/02/20/Cor...
0,Prices may be rising in North Korea as the cou...,https://www.upi.com/Top_News/World-News/2020/0...
0,"Experts, civic groups and others gathered in S...",https://www.upi.com/Top_News/World-News/2020/0...
0,South Korean officials reported the nation's f...,https://www.upi.com/Top_News/World-News/2020/0...
0,Two passengers of a quarantined cruise ship in...,https://www.upi.com/Top_News/World-News/2020/0...
0,North Korea state media berated South Korean t...,https://www.upi.com/Top_News/World-News/2020/0...
0,China is using all available measures to stop ...,https://www.upi.com/Top_News/World-News/2020/0...
0,"Egypt, Algeria and South Africa are among the ...",https://www.upi.com/Health_News/2020/02/19/Egy...
0,South Korea reported a record number of confir...,https://www.upi.com/Top_News/World-News/2020/0...
0,"In my book, Anatomy of Failure: Why America Lo...",https://www.upi.com/Top_News/Voices/2020/02/19...


In [84]:
final_df = pd.concat([df_text, ap_text, rue_text, upi_text])

In [87]:
final_df.reset_index().drop(columns = 'index', inplace = True)

In [88]:
final_df.to_csv('../Data/news_df.csv', index = False)